---
draft: true
title: Finding pole positions
---

In [ ]:
# | code-fold: true
# | code-summary: Import Python libraries
from __future__ import annotations

from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import unevaluated
from IPython.display import Math

## Theory

Define $T$ matrix via $K$-matrix:
$$T(s) = \left(\mathbb{I} + i K(s) {\rho_{cm}(s)}\right)^{-1} K(s)$$
For one channel and multiple resonances this gives:\
$$\sum_\mathrm{Res.}\frac{{K(s)}}{i {K(s)} {{\rho_{cm}(s)}} + 1}$$

A resonance pole in the complex $s$ plane occurs when the denominator of $T(s)$ goes to zero:
$$|T(s)| \rightarrow \infty \quad \Rightarrow \quad \left| \frac{1}{1 - i K(s)} \right|^2 \rightarrow \infty$$

Finding the poles is equivalent to minimizing:

$$\left|1 - i K(s)\right|^2 \rightarrow 0$$

to solve the problem numerically one can separate real and complex part of $s$ to construct a cost function which can be given to common minimizer: Minuit2
Cost function:
$$\text{Cost}(x, y) = \left|1 - i K(x + i y)\right|^2$$
using where $x$ and $y$ are minimization parameters.

## Symbolic implementation

In [ ]:
# @title
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


@unevaluated(real=False)
class PhaseSpaceFactorCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))

In [ ]:
s = sp.symbols("s")
ma, mb = sp.symbols("ma mb")

In [ ]:
parameters = {
    "ma": 0.2,
    "mb": 0.3,
}

In [ ]:
rho_func = sp.lambdify(s, PhaseSpaceFactor(s, ma, mb).doit().subs(parameters))
rho_cm_func = sp.lambdify(s, PhaseSpaceFactorCM(s, ma, mb).doit().subs(parameters))

In [ ]:
x = np.linspace(0, 1, 100)
epsilon = 1e-10j
plt.plot(
    x,
    rho_func(x + epsilon).real,
    color="blue",
    label=r"$\rho(s) real$",
    alpha=0.5,
)
plt.plot(
    x,
    rho_cm_func(x + epsilon).real,
    color="red",
    label=r"$\rho^{CM}(s) real$",
    alpha=0.5,
)
plt.plot(
    x,
    rho_func(x + epsilon).imag,
    color="blue",
    linestyle="--",
    label=r"$\rho(s) imag$",
)
plt.plot(
    x,
    rho_cm_func(x + epsilon).imag,
    color="red",
    linestyle="--",
    label=r"$\rho^{CM}(s) imag$",
)
plt.ylim(0, 2)
plt.legend()
plt.show()

In [ ]:
n = 1
K = sp.MatrixSymbol("K", n, n)
rho_cm = sp.MatrixSymbol("rho_CM", n, n)
rho = sp.MatrixSymbol("rho", n, n)
I = sp.Identity(n)

In [ ]:
T1 = (I - sp.I * rho_cm * K).inv() * K
T1

In [ ]:
T1.as_explicit()

In [ ]:
s, g1, m1, g2, m2 = sp.symbols("s g1 m1 g2 m2")
ma, mb = sp.symbols("ma mb")

In [ ]:
substitutions = {
    K[0, 0]: g1**2 / (m1**2 - s) + g2**2 / (m2**2 - s),
    rho_cm[0, 0]: PhaseSpaceFactorCM(s, ma, mb),
    rho[0, 0]: PhaseSpaceFactor(s, ma, mb),
}
Math(aslatex(substitutions))

In [ ]:
T1_expr = T1.as_explicit().subs(substitutions)[0].simplify(doit=False)
T1_expr

In [ ]:
T2 = (T1.inv() - 2 * sp.I * rho).inv()
T2

In [ ]:
T2_expr = T2.as_explicit().subs(substitutions)[0].simplify(doit=False)
T2_expr

In [ ]:
parameters = {
    "m1": 1.8,
    "m2": 1.1,
    "g1": 0.5,
    "g2": 0.7,
    "ma": 0.1,
    "mb": 0.2,
}

In [ ]:
T1_func = sp.lambdify(s, T1_expr.doit().subs(parameters))
T2_func = sp.lambdify(s, T2_expr.doit().subs(parameters))

In [ ]:
x = np.linspace(0, 5, 500)
y = T2_func(x + 1e-8j)
plt.plot(x, y.real, label="real")
plt.show()

In [ ]:
X, Y = np.meshgrid(
    np.linspace(0, 6, num=100),
    np.linspace(1e-8, 2, num=100),
)
S = X + 1j * Y
mesh = plt.pcolormesh(X, +Y, T1_func(S).imag, vmin=-1, vmax=1)
mesh = plt.pcolormesh(X, -Y, T2_func(S.conj()).imag, vmin=-1, vmax=1)
plt.xlabel("Re(s)")
plt.ylabel("Im(s)")
plt.colorbar(mesh)
plt.show()

In [ ]:
numerator, denominator = sp.fraction(T2_expr)
denominator

In [ ]:
denominator_expr = denominator.doit()

In [ ]:
denominator_func = sp.lambdify(s, denominator_expr.subs(parameters))

In [ ]:
Z = np.abs(denominator_func(S.conj()))
mesh = plt.pcolormesh(X, -Y, Z, vmin=0, vmax=3)
plt.colorbar()

In [ ]:
S.flatten()[Z.flatten() < 0.1]  # noqa: PLR2004

## Find pole positions via Minuit2 fit